In [565]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as ptch
%matplotlib inline
import pandas as pd
from ipywidgets import interact

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.16f}'.format

$f(x) = x\sin(x)\cos(x)$

In [566]:
def F(x: float) -> float:
    return x * np.cos(x) * np.sin(x)

$\displaystyle\int f(x)\ dx = \frac{1}{8}(\sin(2x) - 2x\cos(2x)) + c$

In [567]:
def IF(x: float) -> float:
    return (np.sin(2 * x) - 2 * x * np.cos(2 * x)) / 8

$p_0(x) = 2$


In [568]:
def P0(x: float) -> float:
    return 2

$\displaystyle\int p_0(x)\ dx = 2x + c$ 

In [569]:
def IP0(x: float) -> float:
    return 2 * x

$p_1(x) = x+1$


In [570]:
def P1(x: float) -> float:
    return x + 1

$\displaystyle\int p_1(x)\ dx = \frac{x^2}{2} + x + c$

In [571]:
def IP1(x: float) -> float:
    return x ** 2 / 2 + x

$p_2(x) = 3x^2 - 2$

In [572]:
def P2(x: float) -> float:
    return 3 * x ** 2 - 2

$\displaystyle\int p_2(x)\ dx = x^3 - 2x + c$

In [573]:
def IP2(x: float) -> float:
    return x ** 3 - 2 * x

$p_3(x) = 4x^3 - 3x^2 + 2x$

In [574]:
def P3(x: float) -> float:
    return 4 * x ** 3 - 3 * x ** 2 + 2 * x

$\displaystyle\int p_3(x)\ dx = x^4 - x^3 + x^2$

In [575]:
def IP3(x: float) -> float:
    return x ** 4 - x ** 3 + x ** 2

Адаптированные квадратурные формулы

In [576]:
def getSum(f, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    return sum([f(a + i * h) for i in range(1, n)])


def KFL(f, s: float, a: float, b: float, n: int) -> float:
    return (f(a) + s) * (b - a) / n


def KFR(f, s: float, a: float, b: float, n: int) -> float:
    return (s + f(b)) * (b - a) / n


def KFM(f, s: float, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    return sum([f(a + i * h + h / 2) for i in range(n)]) * (b - a) / n


def KFT(f, s: float, a: float, b: float, n: int) -> float:
    return (f(a) + 2 * s + f(b)) * (b - a) / (2 * n)


def KFS(f, s: float, a: float, b: float, n: int) -> float:
    h = (b - a) / n
    sMid = sum([f(a + (2 * i + 1) * h / 2) for i in range(n)])
    return (f(a) + 2 * s + 4 * sMid + f(b)) * h / 6

Элементы для вывода

In [577]:
def printGraph(f, a: float, b: float, expected: float, n: int, name: str, res: float):

    ptsnum = 1000
    ep = (b - a) / 8

    X = np.linspace(a - ep, b + ep, ptsnum)
    y = [f(x) for x in X]
    Ox = [0 for _ in X]

    fig, ax = plt.subplots()
    ax.plot(X, y, color="blue", alpha=0.5, label=f"{f.__name__}(x)")
    ax.plot(X, Ox, color="black")

    X = [x for x in X if a <= x <= b]
    y = [f(x) for x in X]
    Xaxis = [0 for _ in X]

    h = (b - a) / n

    figures = []

    if (name == "KFL"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c)), (c+h, f(c)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFR"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c + h)), (c+h, f(c + h)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFM"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c + h / 2)), (c+h, f(c + h / 2)), (c+h, 0))
            figures.append(elem)
    elif (name == "KFT"):
        for i in range(n):
            c = a + i * (b - a) / n
            elem = ((c, 0), (c, f(c)), (c+h, f(c + h)), (c+h, 0))
            figures.append(elem)

    ax.add_patch(ptch.Polygon(figures[0], edgecolor="#1207ff", facecolor="#1207ff", 
                                        fill=True, alpha=0.2, label=f"{round(res, 3)}"))       

    figures.remove(figures[0])
    
    [ax.add_patch(ptch.Polygon(figure, edgecolor="#1207ff", facecolor="#1207ff", 
                                        fill=True, alpha=0.2)) for figure in figures]

    ax.fill_between(X, Xaxis, y, facecolor="red", alpha=0.2, label=f"{round(expected, 3)}")
    ax.vlines(a, min(f(a), 0), max(f(a), 0), color="blue", alpha=0.2, linestyle="--")
    ax.vlines(b, min(f(b), 0), max(f(b), 0), color="blue", alpha=0.2, linestyle="--")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.legend()

    plt.show()


def getGraph(f, If, formula, a: float, b: float, n: int):
    Sum = getSum(f, a, b, n)
    Res = formula(f, Sum, a, b, n)
    Expected = If(b) - If(a)
    printGraph(f, a, b, Expected, n, formula.__name__, Res)


def getGraphInteract(f, If, formula, a, b, n__):
    def graph_(n=n__):
        return getGraph(f, If, formula, a, b, n)
    return graph_


def getData(f, If, a: float, b: float, n: int):
    Sum = getSum(f, a, b, n)
    Expected = If(b) - If(a)
    Results = [If(b) - If(a), KFL(f, Sum, a, b, n), KFR(f, Sum, a, b, n), 
                        KFM(f, Sum, a, b, n), KFT(f, Sum, a, b, n), KFS(f, Sum, a, b, n)]
    data = {
        "Значение": Results,
        "Погрешность": [abs(Expected - result) for result in Results]
    }
    df = pd.DataFrame(data, index=["Значение интеграла", "СКФ левых прямоугольников", 
                        "СКФ правых прямоугольников", "СКФ средних прямоугольников",
                        "СКФ трапеций", "СКФ Симпсона"])
    print(df)


Входные данные:

In [578]:
a_ = -6
b_ = 6
n_ = 20

Визуализация СКФ левых прямоугольников для $f(x)$

In [579]:
graph = getGraphInteract(F, IF, KFL, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 
 

interactive(children=(IntSlider(value=20, description='n', max=40, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=20)>

Визуализация СКФ правых прямоугольников для $f(x)$

In [580]:
graph = getGraphInteract(F, IF, KFR, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=20, description='n', max=40, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=20)>

Визуализация СКФ средних прямоугольников для $f(x)$

In [581]:
graph = getGraphInteract(F, IF, KFM, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=20, description='n', max=40, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=20)>

Визуализация СКФ трапеций для $f(x)$

In [582]:
graph = getGraphInteract(F, IF, KFT, a_, b_, n_)
interact(graph, n=(2, n_ * 2, 1)) 

interactive(children=(IntSlider(value=20, description='n', max=40, min=2), Output()), _dom_classes=('widget-in…

<function __main__.getGraphInteract.<locals>.graph_(n=20)>

Значения СКФ и их абсолютная фактическая погрешность для $f(x)$

In [583]:
getData(F, IF, a_, b_, n_)

                                       Значение        Погрешность
Значение интеграла          -2.6657051056975849 0.0000000000000000
СКФ левых прямоугольников   -2.3716933847368438 0.2940117209607411
СКФ правых прямоугольников  -2.3716933847368438 0.2940117209607411
СКФ средних прямоугольников -2.8150994336662274 0.1493943279686425
СКФ трапеций                -2.3716933847368438 0.2940117209607411
СКФ Симпсона                -2.6672974173564366 0.0015923116588517


Значения СКФ и их абсолютная фактическая погрешность для $p_0(x)$

In [584]:
getData(P0, IP0, a_, b_, n_)

                                       Значение        Погрешность
Значение интеграла          24.0000000000000000 0.0000000000000000
СКФ левых прямоугольников   24.0000000000000000 0.0000000000000000
СКФ правых прямоугольников  24.0000000000000000 0.0000000000000000
СКФ средних прямоугольников 24.0000000000000000 0.0000000000000000
СКФ трапеций                24.0000000000000000 0.0000000000000000
СКФ Симпсона                24.0000000000000000 0.0000000000000000


Значения СКФ и их абсолютная фактическая погрешность для $p_1(x)$

In [585]:
getData(P1, IP1, a_, b_, n_)

                                       Значение        Погрешность
Значение интеграла          12.0000000000000000 0.0000000000000000
СКФ левых прямоугольников    8.3999999999999968 3.6000000000000032
СКФ правых прямоугольников  15.5999999999999979 3.5999999999999979
СКФ средних прямоугольников 11.9999999999999964 0.0000000000000036
СКФ трапеций                11.9999999999999964 0.0000000000000036
СКФ Симпсона                11.9999999999999982 0.0000000000000018


Значения СКФ и их абсолютная фактическая погрешность для $p_2(x)$

In [586]:
getData(P2, IP2, a_, b_, n_)

                                        Значение        Погрешность
Значение интеграла          408.0000000000000000 0.0000000000000000
СКФ левых прямоугольников   410.1600000000000250 2.1600000000000250
СКФ правых прямоугольников  410.1600000000000250 2.1600000000000250
СКФ средних прямоугольников 406.9200000000000159 1.0799999999999841
СКФ трапеций                410.1600000000000250 2.1600000000000250
СКФ Симпсона                408.0000000000000000 0.0000000000000000


Значения СКФ и их абсолютная фактическая погрешность для $p_3(x)$

In [587]:
getData(P3, IP3, a_, b_, n_)

                                         Значение          Погрешность
Значение интеграла          -432.0000000000000000   0.0000000000000000
СКФ левых прямоугольников   -959.7600000000005593 527.7600000000005593
СКФ правых прямоугольников    91.4399999999995714 523.4399999999995998
СКФ средних прямоугольников -430.9200000000006412   1.0799999999993588
СКФ трапеций                -434.1600000000004798   2.1600000000004798
СКФ Симпсона                -432.0000000000005116   0.0000000000005116
